# Importing Libraries

In [1]:
import sys
import io
import textblob
from textblob import TextBlob
import tweepy
from tweepy import OAuthHandler
import re as re
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

#TUTORIAL: https://www.datacamp.com/community/tutorials/wordcloud-python

# Get Tweets

In [2]:
# Replace the API_KEY and API_SECRET with your application's key and secret.
ckey = "XjFwLTGuvCYX3PUoPHDoCMbT1"
csecret = "KeIa69uyKAAv6jeCLaT1lC19Y6Q7vI0upsIrVeAMwKZq4yHvmA"
atoken = "228985267-ZiPdBxOnpjMxgGaD1y2pnE1W05UzqjB4RTjD1GCI"
asecret = "WACvIuXKj3quOZf8bK3UjFjgrJzY4Q3lO6kyMhrMR3woQ"

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

searchQuery = "women technology -nail -lash -filter:retweets"  # this is what we're searching for
filterText = ''  # To filter returned tweets based on a second filter
maxTweets = 5000  # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = 0

tweetCount = 0
print("Max of {0} tweets are requested".format(maxTweets))


# To filter returned tweets based on a second filter
def tweetFilter(text, filterText):
    if ((filterText in text)):

        return True
    else:
        return False

fString = "new.csv"
fName = fString.format(tweetCount)  # We'll store the tweets in a text file.
f = io.open(fName, encoding='utf-8', mode='w')


loopMax = 20
loopCounter = 0

f.write('created_at' + ',' + 'full_text' + ',' + 'retweet' + ',' + 
                'favorites' + '\n')

while ((tweetCount < maxTweets) and (loopCounter < loopMax)):
    loopCounter += 1
    try:
        if (not sinceId):
            new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en", max_id=str(max_id - 1), tweet_mode='extended')
        else:
            new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en", max_id=str(max_id - 1),
                                    since_id=sinceId)

        if not new_tweets:
            # print("No more tweets found")
            break

        for tweet in new_tweets:
            if (tweetFilter(tweet._json['full_text'], filterText)):
                f.write(tweet._json['created_at'] + ',' + ((tweet._json['full_text']).replace("\n", " ")).replace(","," ") +
                         ',' + str(tweet._json['retweet_count']) + ',' + str(tweet._json['favorite_count']) + '\n')
                tweetCount += 1

        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

print("Downloaded {0} tweets".format(tweetCount))
f.close()

Max of 5000 tweets are requested
Downloaded 1999 tweets


# Create Pandas Dataframe

In [3]:
df = pd.read_csv(fString)

pd.set_option('max_colwidth', 500)
df.shape

(1954, 4)

In [4]:
df.head()

,created_at,full_text,retweet,favorites
0,Tue Apr 16 19:30:29 +0000 2019,In this edition of JHM Sibley get acquainted with our integrative approach to women's care and holistic well-being. Learn about exciting new additions in imaging technology and sleep research gain insights from our team and view upcoming programs. https://t.co/8u587xKcre https://t.co/DwkLR7xJ2F,0,0
1,Tue Apr 16 19:26:43 +0000 2019,@recursecenter Instead it can be powerful to be clear about what you really mean: Inclusion of “people of color and white women ” which is an explicit acknowledgment of groups currently excluded from participation in technology listed in order of severity. Though even this may be incomplete,0,3
2,Tue Apr 16 19:17:57 +0000 2019,In our efforts to encourage more women to experiment with technology we created the LED Ribbon “Feather Flower” Kit. Now women who love to make and #DIYfashion can add light to… https://t.co/uKUVwrPCGJ,0,1
3,Tue Apr 16 19:00:41 +0000 2019,We kicked up the style quotient with bold colours glossy uppers and a slimmer more streamlined silhouette you can wear all day. Expect the same OOFOAM™ technology as the OOriginal with a narrower footbed designed specifically for women. https://t.co/yPzcFA3tBp #ukrunchat https://t.co/bidmslq5nf,0,0
4,Tue Apr 16 18:58:42 +0000 2019,The Technology Safety and Privacy Toolkit is an online toolkit for Canadian women experiencing technology-facilitated violence to learn how they can increase their technology safety and privacy. cc @landvdotnet https://t.co/pCVOO4q0JH,0,1


# Clean Tweets

In [5]:
df['clean_tweet'] = df['full_text'].map(lambda x: re.sub(r'[Ä|ô|ô|•|Ô|∏|è]+', '', x))
df['clean_tweet'] = df['clean_tweet'].map(lambda x: re.sub(r'‚', '\'', x))
df['clean_tweet'] = df['clean_tweet'].map(lambda x: re.sub(r'\&amp\;', '&', x))
df['clean_tweet'] = df['clean_tweet'].map(lambda x: re.sub(r'\&gt\;', '>', x))
df['clean_tweet'] = df['clean_tweet'].map(lambda x: re.sub(r'\&lt\;', '<', x))

df.head()


,created_at,full_text,retweet,favorites,clean_tweet
0,Tue Apr 16 19:30:29 +0000 2019,In this edition of JHM Sibley get acquainted with our integrative approach to women's care and holistic well-being. Learn about exciting new additions in imaging technology and sleep research gain insights from our team and view upcoming programs. https://t.co/8u587xKcre https://t.co/DwkLR7xJ2F,0,0,In this edition of JHM Sibley get acquainted with our integrative approach to women's care and holistic well-being. Learn about exciting new additions in imaging technology and sleep research gain insights from our team and view upcoming programs. https://t.co/8u587xKcre https://t.co/DwkLR7xJ2F
1,Tue Apr 16 19:26:43 +0000 2019,@recursecenter Instead it can be powerful to be clear about what you really mean: Inclusion of “people of color and white women ” which is an explicit acknowledgment of groups currently excluded from participation in technology listed in order of severity. Though even this may be incomplete,0,3,@recursecenter Instead it can be powerful to be clear about what you really mean: Inclusion of “people of color and white women ” which is an explicit acknowledgment of groups currently excluded from participation in technology listed in order of severity. Though even this may be incomplete
2,Tue Apr 16 19:17:57 +0000 2019,In our efforts to encourage more women to experiment with technology we created the LED Ribbon “Feather Flower” Kit. Now women who love to make and #DIYfashion can add light to… https://t.co/uKUVwrPCGJ,0,1,In our efforts to encourage more women to experiment with technology we created the LED Ribbon “Feather Flower” Kit. Now women who love to make and #DIYfashion can add light to… https://t.co/uKUVwrPCGJ
3,Tue Apr 16 19:00:41 +0000 2019,We kicked up the style quotient with bold colours glossy uppers and a slimmer more streamlined silhouette you can wear all day. Expect the same OOFOAM™ technology as the OOriginal with a narrower footbed designed specifically for women. https://t.co/yPzcFA3tBp #ukrunchat https://t.co/bidmslq5nf,0,0,We kicked up the style quotient with bold colours glossy uppers and a slimmer more streamlined silhouette you can wear all day. Expect the same OOFOAM™ technology as the OOriginal with a narrower footbed designed specifically for women. https://t.co/yPzcFA3tBp #ukrunchat https://t.co/bidmslq5nf
4,Tue Apr 16 18:58:42 +0000 2019,The Technology Safety and Privacy Toolkit is an online toolkit for Canadian women experiencing technology-facilitated violence to learn how they can increase their technology safety and privacy. cc @landvdotnet https://t.co/pCVOO4q0JH,0,1,The Technology Safety and Privacy Toolkit is an online toolkit for Canadian women experiencing technology-facilitated violence to learn how they can increase their technology safety and privacy. cc @landvdotnet https://t.co/pCVOO4q0JH


In [6]:
dateTime=df['created_at'].str.split(' ',n=6,expand=True)
df['day']=dateTime[0]
df['month']=dateTime[1]
df['dayNum']=dateTime[2]
df['time']=dateTime[3]
df['year']=dateTime[5]

In [7]:
month={'Mar':3,'Apr':4}
df['month']=[month[m] for m in df.month]

weekday={'Sun':0,'Mon':1,'Tue':2,'Wed':3,'Thu':4,'Fri':5,'Sat':6}
df['weekday']=df['day']
df.day=[weekday[d] for d in df.day] 

# Sentiment Analysis

In [8]:
def polarity(text):
    return (TextBlob(text).sentiment)

def subjectivity(text):
    return (TextBlob(text).sentiment.subjectivity)

def posNB(text):
    blob=TextBlob(text, analyzer=NaiveBayesAnalyzer())
    return (blob.sentiment.p_pos)

def negNB(text):
    blob=TextBlob(text, analyzer=NaiveBayesAnalyzer())
    return (blob.sentiment.p_neg)

df['sentiment_polarity'] = df['clean_tweet'].apply(polarity)
df['sentiment_subjectivity'] = df['clean_tweet'].apply(subjectivity)

dfOriginal=df

# Describe Data

In [10]:
df.describe()

,retweet,favorites,day,month,sentiment_subjectivity
count,1954.000000,1954.000000,1954.000000,1954.0,1954.000000
mean,2.068066,6.750768,3.186285,4.0,0.410699
std,14.462833,58.857625,1.805180,0.0,0.280571
min,0.000000,0.000000,0.000000,4.0,0.000000
25%,0.000000,0.000000,2.000000,4.0,0.166667
50%,0.000000,1.000000,3.000000,4.0,0.426627
75%,1.000000,3.000000,5.000000,4.0,0.600000
max,499.000000,1777.000000,6.000000,4.0,1.000000


In [ ]:
df.plot(kind='scatter',x='sentiment_polarity',y='favorites',color='red')

In [ ]:
df.plot(kind='scatter',x='sentiment_polarity',y='retweets',color='red')

In [ ]:
df.plot(kind='scatter',x='sentiment_subjectivity',y='sentiment_polarity',color='red')

In [ ]:
df.plot(kind='scatter',x='sentiment_subjectivity',y='retweets',color='red')

# Removing Outliers

In [ ]:
#Identify Outliers

#Favorites
df.loc[df['favorites']>6000]

In [ ]:
mostPopular = df.iloc[0]['full_text']

tb = Blobber(analyzer=NaiveBayesAnalyzer())

print(tb("sentence you want to test"))

pol=posNB(mostPopular)
print("Pos Polarity: " + str(pol))
pol=negNB(mostPopular)
print("Neg Polarity: " + str(pol))

popularWords = mostPopular.split()

#for w in popularWords:
#    print("Word: " + w)
#    pol=polarity2(w)
#    print("Polarity: " + str(pol))
    #sub=subjectivity(w)
    #print("Subjectivity: " + str(sub))

In [ ]:
df=df.drop(0)
df=df.drop(1)

In [ ]:
df.plot(kind='scatter',x='sentiment_polarity',y='retweets',color='red')

In [ ]:
df.loc[df['retweets']>2000]

In [ ]:
df=df.drop(2)
df.plot(kind='scatter',x='sentiment_polarity',y='retweets',color='red')

In [ ]:
df.describe()

In [ ]:
dfNeut=df.loc[df['sentiment_polarity']==0]
dfNeut.describe()

In [ ]:
dfNeg=df.loc[df['sentiment_polarity']<0]
dfNeg.describe()

In [ ]:
dfNeg.plot(kind='scatter',x='sentiment_polarity',y='retweets',color='red')

In [ ]:
dfPos=df.loc[df['sentiment_polarity']>0]
dfPos.describe()

In [ ]:
dfPos.plot(kind='scatter',x='sentiment_polarity',y='retweets',color='red')

In [ ]:
fig5, box = plt.subplots()
box.boxplot(df['sentiment_subjectivity'], vert=False)

In [ ]:
fig5, box = plt.subplots()
box.boxplot(df['sentiment_polarity'], vert=False)

# Keywords

In [ ]:
dfWomen=df[df['clean_tweet'].str.contains('women')]

dfFemale=df[df['clean_tweet'].str.contains('female')]

dfGirl=df[df['clean_tweet'].str.contains('girl')]

In [ ]:
#https://machinelearningmastery.com/parametric-statistical-significance-tests-in-python/

from scipy.stats import f_oneway

# compare samples
stat, p = f_oneway(dfWomen['sentiment_polarity'], dfFemale['sentiment_polarity'], dfGirl['sentiment_polarity'])
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distributions (fail to reject H0)')
else:
	print('Different distributions (reject H0)')

In [ ]:
print('Women')
print(dfWomen['sentiment_polarity'].mean())
print('Female')
print(dfFemale['sentiment_polarity'].mean())
print('Girl')
print(dfGirl['sentiment_polarity'].mean())

In [ ]:
dfWomen.plot(kind='scatter',x='sentiment_polarity',y='favorites',color='red')

In [ ]:
dfFemale.plot(kind='scatter',x='sentiment_polarity',y='favorites',color='green')

In [ ]:
dfGirl.plot(kind='scatter',x='sentiment_polarity',y='favorites',color='blue')

# Polarity

In [ ]:
fig5, box = plt.subplots()
box.boxplot(df['sentiment_polarity'], vert=False)

# Date/Time

In [ ]:
df2=df.groupby('dayNum').mean()[['sentiment_polarity']]
df2.reset_index()
df2['dayNum']=df2.index

df2.plot(kind='line',x='dayNum',y='sentiment_polarity',color='red')

In [ ]:
print("March 3: Sunday")
print("March 19: Tuesday")
print("March 20: Wednesday")
print("March 23: Saturday")
print("March 24: Sunday")
print("March 27: Wednesday")

In [ ]:
df2.describe()

In [ ]:
df = df.sort_values(by="sentiment_polarity")
df = df.reset_index(drop=True)
df.loc[df['dayNum']=="03"]

In [ ]:
df2=df.groupby('day').mean()[['sentiment_polarity']]
df2.reset_index()
df2['day']=df2.index
df2.plot(kind='line',x='day',y='sentiment_polarity',color='red')

In [ ]:
df.loc[df['day']==3]

# Wordcloud

In [ ]:
text = " ".join(tweet for tweet in df.full_text)
#text = " ".join(tweet for tweet in df.full_text)
print ("There are {} words in the combination of all review.".format(len(text)))

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["https", "co","amp","Äô","Äôs","Äù"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
stopwords.update(["https", "co","amp","Äô","Äôs","Äôt","Äù","women","girl","female"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
text = " ".join(tweet for tweet in df.full_text[df['sentiment_polarity']<0])
#text = " ".join(tweet for tweet in df.full_text)
print ("There are {} words in the combination of all review.".format(len(text)))
stopwords.update(["https", "co","amp","Äô","Äôs","Äù","Äôt",
                  "women","girl","female", "tech", "technology",
                  "computer","men","one","know","people","will"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()